In [1]:
#imports
import pandas as pd
import numpy as np
from collections import Counter
import re
import csv
import seaborn as sns
import matplotlib.pylab as plt
from scipy.stats import pearsonr, spearmanr

import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='camligorkem', api_key='ls8JEHoNuI6dj5AHrzy2')

In [3]:
topic_distribution = pd.read_csv('../Data/topic_distribution.csv')
full_data_sources = pd.read_csv('../Data/final-news-data.csv')
all_facts_org = pd.read_csv('../Data/AllFacts.csv')

In [4]:
topic_distribution.head()

,country,country_name,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,US,United States,0.022056,NaN,0.194978,NaN,0.113442,0.082864,0.132510,0.055243,0.084825,0.084809,0.084572,0.056618,0.088082
1,CA,Canada,0.194973,NaN,NaN,NaN,0.095840,NaN,0.146600,0.073039,0.073039,0.138273,0.165221,NaN,0.113016
2,GB,United Kingdom,NaN,0.175466,NaN,NaN,0.228910,NaN,0.057249,0.147708,0.148592,0.037203,NaN,0.121004,0.083868
3,IE,Ireland,NaN,NaN,0.177962,NaN,0.252041,NaN,0.054292,0.184377,NaN,0.143317,NaN,0.133860,0.054152
4,AU,Australia,NaN,0.029435,NaN,NaN,0.124271,0.163525,0.112324,0.153065,NaN,0.157716,NaN,0.117698,0.141965


In [5]:
full_data_sources.head()

,textID,#words,date,country,website,url,title,Topics
0,3732490,815,2015-11-01,US,NPR,http://www.npr.org/2015/11/01/450889721/the-ma...,The Madonna Of 115th Street Gets A Long-Awaite...,SOCIAL_LIFE/DAILY
1,3732492,258,2015-11-01,US,Huffington Post,http://www.huffingtonpost.com/entry/university...,University Of Louisville Sorry A Bunch Of Its ...,SOCIAL_LIFE/DAILY
2,3732496,489,2015-11-01,US,Bleacher Report,http://bleacherreport.com/articles/2584708-kar...,'Kareem: Minority of One' HBO Documentary Prev...,POLITICS
3,3732501,840,2015-11-01,US,VentureBeat,http://venturebeat.com/2015/11/01/what-big-ind...,What big industry will do to the Internet of T...,HEALTH/MEDICAL
4,3732502,470,2015-11-01,US,Tech Insider,http://www.techinsider.io/daenerys-game-of-thr...,Daenerys has been traveling in the wrong direc...,SPORTS


In [6]:
all_facts_org.head(2)

,Country,Population,Age_structure0-14,Age_structure15-24,Age_structure25-54,Age_structure55-64,Age_structureover65,Median_age,Population_growth_rate,Birth_rate,...,Sex_ratio,Life_expectancy_at_birth,GDP_per_capita,Unemployment_rate,Inflation_rate,Electricity_renewable_sources,Carbon_dioxide_emissions,Internet_users,Religions,Ethnic_groups
0,United States,323995528,18.84,13.46,39.60,12.85,15.25,37.9,0.81,12.5,...,0.97,79.8,57300,4.7,1.3,7.4,5402.0,74.6,"Protestant 46.5%, Roman Catholic 20.8%, Mormon...","white 79.96%, black 12.85%, Asian 4.43%, Ameri..."
1,Ireland,4952473,21.51,11.80,43.52,10.33,12.84,36.4,1.20,14.5,...,1.00,80.8,69400,8.0,0.2,25.0,34.0,80.1,"Roman Catholic 84.7%, Church of Ireland 2.7%, ...","Irish 84.5%, other white 9.8%, Asian 1.9%, bla..."


In [7]:
cols = all_facts_org.columns
cols[1:-2]

Index(['Population', 'Age_structure0-14', 'Age_structure15-24',
       'Age_structure25-54', 'Age_structure55-64', 'Age_structureover65',
       'Median_age', 'Population_growth_rate', 'Birth_rate', 'Death_rate',
       'Net_migration_rate', 'Sex_ratio', 'Life_expectancy_at_birth',
       'GDP_per_capita', 'Unemployment_rate', 'Inflation_rate',
       'Electricity_renewable_sources', 'Carbon_dioxide_emissions',
       'Internet_users'],
      dtype='object')

In [8]:
all_facts_std = all_facts_org.copy()
all_facts_std[cols[1:-2]] = all_facts_std[cols[1:-2]].apply(lambda x: (x - np.mean(x)) / (np.std(x)))


In [9]:
all_facts_std

,Country,Population,Age_structure0-14,Age_structure15-24,Age_structure25-54,Age_structure55-64,Age_structureover65,Median_age,Population_growth_rate,Birth_rate,...,Sex_ratio,Life_expectancy_at_birth,GDP_per_capita,Unemployment_rate,Inflation_rate,Electricity_renewable_sources,Carbon_dioxide_emissions,Internet_users,Religions,Ethnic_groups
0,United States,0.706372,-0.814154,-0.904887,0.070152,1.214698,1.153125,0.956782,-0.745739,-0.794542,...,-0.520573,0.755736,1.168560,-0.470022,-0.622597,0.143751,4.087505,0.710816,"Protestant 46.5%, Roman Catholic 20.8%, Mormon...","white 79.96%, black 12.85%, Asian 4.43%, Ameri..."
1,Ireland,-0.461200,-0.533846,-1.404259,0.900961,0.507516,0.697732,0.773139,-0.132803,-0.549124,...,0.189299,0.873636,1.640903,-0.127622,-0.859982,2.765532,-0.403794,0.906437,"Roman Catholic 84.7%, Church of Ireland 2.7%, ...","Irish 84.5%, other white 9.8%, Asian 1.9%, bla..."
2,Australia,-0.395180,-0.919139,-1.055300,0.483437,0.925651,1.260831,1.042482,-0.368547,-0.843626,...,0.425924,1.038695,0.836749,-0.355889,-0.601016,0.173544,-0.110120,1.066491,"Protestant 30.1% (Anglican 17.1%, Uniting Chur...","English 25.9%, Australian 25.4%, Irish 7.5%, S..."
3,United Kingdom,-0.243534,-0.961133,-1.298970,0.311765,0.911620,1.653867,1.275097,-1.185795,-0.843626,...,-0.047325,0.861846,0.590819,-0.428519,-0.795240,0.858782,0.043244,1.329690,"Christian (includes Anglican, Roman Catholic, ...","white 87.2%, black/African/Caribbean/black Bri..."
4,Canada,-0.349910,-1.171101,-1.307995,0.222750,1.520583,1.706775,1.458741,-0.855753,-1.064502,...,-0.283949,1.003325,0.735254,-0.221004,-0.557855,0.277819,0.039646,1.205204,"Catholic 39% (includes Roman Catholic 38.8%, o...","Canadian 32.2%, English 19.8%, French 15.5%, S..."
5,India,4.156970,0.117058,0.457858,0.347795,-0.342787,-0.577743,-0.304236,-0.148519,0.039880,...,2.082294,-0.576529,-0.806691,-0.086119,0.305364,0.888575,1.146574,-1.017763,"Hindu 79.8%, Muslim 14.2%, Christian 2.3%, Sik...","Indo-Aryan 72%, Dravidian 25%, Mongoloid and o..."
6,New Zealand,-0.462949,-0.717569,-0.874804,0.165526,0.891976,1.094547,0.944539,-0.761455,-0.696375,...,-0.047325,0.920795,0.380021,-0.428519,-0.773660,1.395056,-0.401284,1.194534,"Christian 44.3% (Catholic 11.6%, Anglican 10.8...","European 71.2%, Maori 14.1%, Asian 11.3%, Paci..."
7,South Africa,-0.280605,0.183198,0.481924,0.460124,-0.542033,-0.676002,-0.402179,-0.462845,0.187132,...,-0.283949,-1.213186,-0.552953,1.823021,0.499588,-0.854313,-0.028962,-0.096566,"Protestant 36.6% (Zionist Christian 11.1%, Pen...","black African 80.2%, white 8.4%, colored 8.8%,..."
8,Sri Lanka,-0.397953,-0.235690,-0.531862,0.517348,0.384039,0.038264,0.295666,-0.761455,-0.426414,...,-0.757198,0.402037,-0.631026,-0.490773,0.024818,-0.675556,-0.418855,-0.875494,"Buddhist (official) 70.2%, Hindu 12.6%, Muslim...","Sinhalese 74.9%, Sri Lankan Tamil 11.2%, Sri L..."
9,Singapore,-0.458165,-1.433563,0.166056,2.369711,0.510322,0.017479,0.516038,0.904473,-1.297650,...,-0.757198,1.368814,2.331851,-0.739792,-1.075787,-0.377626,-0.251519,0.977572,"Buddhist 33.9%, Muslim 14.3%, Taoist 11.3%, Ca...","Chinese 74.2%, Malay 13.3%, Indian 9.2%, other..."


In [10]:
all_facts_std.Inflation_rate.describe()

count    2.000000e+01
mean     1.401657e-16
std      1.025978e+00
min     -1.075787e+00
25%     -6.064113e-01
50%     -3.528407e-01
75%      3.053637e-01
max      2.938181e+00
Name: Inflation_rate, dtype: float64

In [11]:
all_facts_cat= all_facts_std.copy()

In [12]:
#cols_to_cat

In [13]:
#labels = [1,2,3]
##cols_to_cat=['Carbon_dioxide_emissions','Electricity_renewable_sources','GDP_per_capita',
##             'Unemployment_rate','Inflation_rate','Life_expectancy_at_birth','Internet_users']
#cols_to_cat = all_facts_org.columns[1:-2]
#for col in cols_to_cat:
#    all_facts_cat[col] = pd.cut(all_facts_org[col],len(labels), labels=labels)
#all_facts_cat


In [14]:
#all_facts = all_facts_cat
all_facts = all_facts_org

In [15]:
all_facts.loc[all_facts['Country']=='Tanzania','Unemployment_rate']=None

In [16]:
TOPIC_LIST = ['ENVIRONMENT/ENERGY','INTERNATIONAL','POLITICS', \
              'SPORTS', 'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY', \
              'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY', \
              'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW','HEALTH/MEDICAL']

In [17]:
# Correlation 
merged_data = topic_distribution.merge(all_facts, left_on='country_name', right_on='Country')
merged_data.head(2)
data = merged_data

In [18]:
data.columns

Index(['country', 'country_name', 'ENVIRONMENT/ENERGY', 'INTERNATIONAL',
       'POLITICS', 'EDUCATION/FAMILY', 'SPORTS',
       'TECHNOLOGY/SCIENCE/SOCIAL MEDIA', 'SOCIAL_LIFE/DAILY',
       'ENTERTAINMENT/ART/MAGAZINE', 'COMPANY/BUSINESS', 'ECONOMY',
       'POLICE/ACCIDENT/VIOLENCE', 'LEGAL/LAW', 'HEALTH/MEDICAL', 'Country',
       'Population', 'Age_structure0-14', 'Age_structure15-24',
       'Age_structure25-54', 'Age_structure55-64', 'Age_structureover65',
       'Median_age', 'Population_growth_rate', 'Birth_rate', 'Death_rate',
       'Net_migration_rate', 'Sex_ratio', 'Life_expectancy_at_birth',
       'GDP_per_capita', 'Unemployment_rate', 'Inflation_rate',
       'Electricity_renewable_sources', 'Carbon_dioxide_emissions',
       'Internet_users', 'Religions', 'Ethnic_groups'],
      dtype='object')

In [19]:
fact_columns = []
for i in data.columns:
    if(i not in TOPIC_LIST and i not in ['country','country_name','Country','EDUCATION/FAMILY']):
        fact_columns.append(i)  
fact_columns

['Population',
 'Age_structure0-14',
 'Age_structure15-24',
 'Age_structure25-54',
 'Age_structure55-64',
 'Age_structureover65',
 'Median_age',
 'Population_growth_rate',
 'Birth_rate',
 'Death_rate',
 'Net_migration_rate',
 'Sex_ratio',
 'Life_expectancy_at_birth',
 'GDP_per_capita',
 'Unemployment_rate',
 'Inflation_rate',
 'Electricity_renewable_sources',
 'Carbon_dioxide_emissions',
 'Internet_users',
 'Religions',
 'Ethnic_groups']

In [20]:
# Topic Distribution per country
traces=[]
for topic in TOPIC_LIST:
    trace1 = go.Bar(
        x=topic_distribution.country_name,
        y=topic_distribution[topic],
        name=topic
    )
    traces.append(trace1)
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='stacked-bar',colorscale='Light24')
###TODO:yüzdelik hale çevirip round edelim

In [21]:
data.describe()

,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,...,Death_rate,Net_migration_rate,Sex_ratio,Life_expectancy_at_birth,GDP_per_capita,Unemployment_rate,Inflation_rate,Electricity_renewable_sources,Carbon_dioxide_emissions,Internet_users
count,13.000000,12.000000,14.000000,0.0,16.000000,12.000000,18.000000,14.000000,13.000000,15.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,19.000000,20.000000,20.000000,20.000000,20.000000
mean,0.103932,0.115086,0.140971,NaN,0.141445,0.125966,0.121846,0.132206,0.103798,0.115236,...,7.250000,1.145000,0.992000,73.390000,27365.000000,9.768421,4.185000,6.435000,516.615000,54.615000
std,0.070760,0.069292,0.096827,NaN,0.053675,0.075115,0.052363,0.054853,0.063879,0.051646,...,1.910773,4.040124,0.043359,8.702141,26282.479155,9.853372,4.754198,6.887385,1226.249218,28.845966
min,0.014033,0.017842,0.033522,NaN,0.070796,0.028847,0.054292,0.042077,0.017421,0.020969,...,3.500000,-4.500000,0.870000,53.400000,3100.000000,2.100000,-0.800000,0.000000,10.000000,5.400000
25%,0.027872,0.064097,0.073536,NaN,0.106116,0.076383,0.104820,0.100629,0.055618,0.075490,...,6.350000,-1.325000,0.970000,67.425000,5700.000000,4.800000,1.375000,0.625000,29.500000,29.000000
50%,0.111274,0.114375,0.114340,NaN,0.123362,0.112088,0.117008,0.134671,0.084825,0.138273,...,7.150000,-0.200000,0.990000,74.300000,12200.000000,5.800000,2.550000,4.900000,93.500000,49.650000
75%,0.141424,0.155383,0.175151,NaN,0.172863,0.167227,0.133094,0.151726,0.125539,0.156390,...,7.900000,2.850000,1.010000,80.900000,46850.000000,8.200000,5.600000,11.375000,409.250000,82.725000
max,0.234718,0.256281,0.347152,NaN,0.252041,0.291896,0.298739,0.243091,0.224668,0.176033,...,12.700000,13.600000,1.080000,85.000000,87100.000000,40.000000,17.800000,25.000000,5402.000000,92.000000


In [22]:
data.isnull().any()

country                            False
country_name                       False
ENVIRONMENT/ENERGY                  True
INTERNATIONAL                       True
POLITICS                            True
EDUCATION/FAMILY                    True
SPORTS                              True
TECHNOLOGY/SCIENCE/SOCIAL MEDIA     True
SOCIAL_LIFE/DAILY                   True
ENTERTAINMENT/ART/MAGAZINE          True
COMPANY/BUSINESS                    True
ECONOMY                             True
POLICE/ACCIDENT/VIOLENCE            True
LEGAL/LAW                           True
HEALTH/MEDICAL                      True
Country                            False
Population                         False
Age_structure0-14                  False
Age_structure15-24                 False
Age_structure25-54                 False
Age_structure55-64                 False
Age_structureover65                False
Median_age                         False
Population_growth_rate             False
Birth_rate      

In [23]:
correlations = pd.DataFrame(fact_columns[:-2]) 
for col in TOPIC_LIST:
    correlations[col] = None
correlations.rename(columns={0:'facts'},inplace=True)
correlations.head()

,facts,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,Population,None,None,None,None,None,None,None,None,None,None,None,None
1,Age_structure0-14,None,None,None,None,None,None,None,None,None,None,None,None
2,Age_structure15-24,None,None,None,None,None,None,None,None,None,None,None,None
3,Age_structure25-54,None,None,None,None,None,None,None,None,None,None,None,None
4,Age_structure55-64,None,None,None,None,None,None,None,None,None,None,None,None


In [24]:
correlations_res = []
for topics in TOPIC_LIST:
    for fact in fact_columns:
        if(fact not in ['Religions','Ethnic_groups']):
            dfx = data[[topics,fact]].copy()
            dfx.dropna(inplace=True)
            corr,pvalue = spearmanr(dfx[topics],dfx[fact])
            #if(topics== 'EDUCATION/FAMILY'):
            #    print(fact,topics,corr)
            correlations.loc[correlations['facts']==fact,[topics]] = corr
            #topics_distribution.loc[topics_distribution['country'] == country, topic] = freq
            if(abs(corr) > 0.25):
                if(pvalue < 0.05):
                    correlations_res.append('***'+fact +' - '+topics+' correlation: '+ str(corr)+' pvalue: '+str(pvalue))
                elif(pvalue < 0.08):
                    correlations_res.append('*'+fact +' - '+topics+' correlation: '+ str(corr)+' pvalue: '+str(pvalue))
                else:
                    correlations_res.append(fact +' - '+topics+' correlation: '+ str(corr)+' pvalue: '+str(pvalue))
                
                #print ('Pearson Correlation: corr_val:' ,corr,' p_value: ',pvalue)

In [25]:
correlations_res =  pd.DataFrame(correlations_res)
correlations_res.to_csv('Correlation_Results.txt',index=False)
correlations_res

,0
0,Age_structureover65 - ENVIRONMENT/ENERGY corre...
1,Population_growth_rate - ENVIRONMENT/ENERGY co...
2,Population_growth_rate - INTERNATIONAL correla...
3,Death_rate - INTERNATIONAL correlation: -0.304...
4,Net_migration_rate - INTERNATIONAL correlation...
5,Sex_ratio - INTERNATIONAL correlation: -0.2736...
6,Population - POLITICS correlation: 0.454945054...
7,Age_structure15-24 - POLITICS correlation: -0....
8,*Net_migration_rate - POLITICS correlation: 0....
9,Sex_ratio - POLITICS correlation: 0.4022160834...


In [26]:
display(correlations)

,facts,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,COMPANY/BUSINESS,ECONOMY,POLICE/ACCIDENT/VIOLENCE,LEGAL/LAW,HEALTH/MEDICAL
0,Population,-0.0494505,0.167832,0.454945,-0.388235,-0.272727,0.170279,-0.569231,0.291209,0.189286,-0.00699301,-0.459341,-0.153846
1,Age_structure0-14,-0.0769231,-0.153846,-0.142857,-0.25,0.013986,0.0196078,0.0725275,-0.318681,0.325,-0.13986,-0.0197802,-0.0699301
2,Age_structure15-24,-0.247253,0.202797,-0.318681,-0.344118,-0.020979,0.467492,-0.0197802,-0.269231,0.164286,0.20979,-0.0857143,0.0559441
3,Age_structure25-54,0.032967,0.0979021,0.116484,0.352941,0.027972,-0.145511,0.134066,0.373626,-0.0785714,0.034965,0.0857143,-0.0559441
4,Age_structure55-64,0.247253,0.0909091,0.23956,0.235294,-0.0769231,-0.120743,-0.208791,0.247253,-0.15,0.013986,0.0813187,0.258741
5,Age_structureover65,0.252747,0.153846,0.120879,0.261765,-0.0699301,-0.147575,-0.0813187,0.186813,-0.185714,0,0.032967,0.328671
6,Median_age,0.175824,0.13986,0.231023,0.250184,-0.0629371,-0.181724,-0.142857,0.269231,-0.18588,-0.027972,0.107692,0.314685
7,Population_growth_rate,-0.255846,-0.409808,0.178022,-0.286976,0.507882,0.122934,0.107692,-0.352132,0.0964286,0.0560421,0.019802,-0.629371
8,Birth_rate,-0.0769231,-0.210158,-0.0637363,-0.211921,0.00699301,-0.0382034,0.0968097,-0.351648,0.321716,-0.202797,-0.0836084,-0.0875658
9,Death_rate,-0.10989,-0.304729,0.125275,0.102941,-0.276708,-0.31079,-0.178218,-0.032967,-0.00714924,-0.237762,-0.428571,0.503497


In [27]:
corr_result = correlations.drop(columns=['facts'])
res= corr_result.values
len(res)
#res

19

In [28]:
trace = go.Heatmap(z=res,
                   x=TOPIC_LIST,
                   y=fact_columns[:-2])
py.iplot([trace], filename='labelled-heatmap')

In [29]:
# XXXX TO DO XXXX education family sil ust tablo
# XXX TO DO XXXXX kategorical yapmaya karar verdik acıkla yukarda
# XXXX

In [30]:
# correlations = pd.DataFrame(fact_columns+TOPIC_LIST) 
# for col in fact_columns+TOPIC_LIST:
#    correlations[col] = None
# correlations.rename(columns={0:'corr'},inplace=True)
# print(correlations.shape)
# correlations.head()


In [31]:
data

,country,country_name,ENVIRONMENT/ENERGY,INTERNATIONAL,POLITICS,EDUCATION/FAMILY,SPORTS,TECHNOLOGY/SCIENCE/SOCIAL MEDIA,SOCIAL_LIFE/DAILY,ENTERTAINMENT/ART/MAGAZINE,...,Sex_ratio,Life_expectancy_at_birth,GDP_per_capita,Unemployment_rate,Inflation_rate,Electricity_renewable_sources,Carbon_dioxide_emissions,Internet_users,Religions,Ethnic_groups
0,US,United States,0.022056,NaN,0.194978,NaN,0.113442,0.082864,0.132510,0.055243,...,0.97,79.8,57300,4.7,1.3,7.4,5402.0,74.6,"Protestant 46.5%, Roman Catholic 20.8%, Mormon...","white 79.96%, black 12.85%, Asian 4.43%, Ameri..."
1,CA,Canada,0.194973,NaN,NaN,NaN,0.095840,NaN,0.146600,0.073039,...,0.98,81.9,46200,7.1,1.6,8.3,564.0,88.5,"Catholic 39% (includes Roman Catholic 38.8%, o...","Canadian 32.2%, English 19.8%, French 15.5%, S..."
2,GB,United Kingdom,NaN,0.175466,NaN,NaN,0.228910,NaN,0.057249,0.147708,...,0.99,80.7,42500,5.1,0.5,12.2,568.3,92.0,"Christian (includes Anglican, Roman Catholic, ...","white 87.2%, black/African/Caribbean/black Bri..."
3,IE,Ireland,NaN,NaN,0.177962,NaN,0.252041,NaN,0.054292,0.184377,...,1.00,80.8,69400,8.0,0.2,25.0,34.0,80.1,"Roman Catholic 84.7%, Church of Ireland 2.7%, ...","Irish 84.5%, other white 9.8%, Asian 1.9%, bla..."
4,AU,Australia,NaN,0.029435,NaN,NaN,0.124271,0.163525,0.112324,0.153065,...,1.01,82.2,48800,5.8,1.4,7.6,385.0,84.6,"Protestant 30.1% (Anglican 17.1%, Uniting Chur...","English 25.9%, Australian 25.4%, Irish 7.5%, S..."
5,NZ,New Zealand,0.141424,NaN,NaN,NaN,0.185989,NaN,0.102319,0.133849,...,0.99,81.2,37100,5.1,0.6,15.8,37.0,88.2,"Christian 44.3% (Catholic 11.6%, Anglican 10.8...","European 71.2%, Maori 14.1%, Asian 11.3%, Paci..."
6,IN,India,NaN,0.017842,0.347152,NaN,0.070796,0.119554,NaN,0.132678,...,1.08,68.5,6700,8.4,5.6,12.4,1887.0,26.0,"Hindu 79.8%, Muslim 14.2%, Christian 2.3%, Sik...","Indo-Aryan 72%, Dravidian 25%, Mongoloid and o..."
7,LK,Sri Lanka,0.234718,0.137635,0.062725,NaN,0.122453,0.113064,0.115661,NaN,...,0.96,76.8,11200,4.5,4.3,1.9,16.0,30.0,"Buddhist (official) 70.2%, Hindu 12.6%, Muslim...","Sinhalese 74.9%, Sri Lankan Tamil 11.2%, Sri L..."
8,PK,Pakistan,NaN,0.256281,0.321481,NaN,0.071147,0.028847,0.146365,0.042077,...,1.06,67.7,5100,6.7,3.7,0.4,145.0,18.0,"Muslim (official) 96.4% (Sunni 85-90%, Shia 10...","Punjabi 44.68%, Pashtun (Pathan) 15.42%, Sindh..."
9,BD,Bangladesh,0.157019,0.175870,0.034819,NaN,NaN,0.111111,0.133289,NaN,...,0.97,73.2,3900,4.9,5.6,0.0,66.0,14.4,"Muslim 89.1%, Hindu 10%, other 0.9% (includes ...","Bengali at least 98%, ethnic groups 1.1%"


In [44]:
countryColors = 
# Net_migration rate - Politics
trace = go.Scatter(
    x = data.Net_migration_rate,
    y = data.POLITICS,
    mode = 'markers',
    text = data.Country,
    textposition = 'bottom center',
    marker={
            "color": ["#bebada","#fdb462","#fb8072","#d9d9d9","#bc80bd","#b3de69","#8dd3c7","#80b1d3","#fccde5","#ffffb3"],
             "size": 10
            }
)
# Plot and embed in ipython notebook!
py.iplot([trace], filename='net-migration-vs-politics')

## Unique Source Distribution  ---  Data Collection Bias

In [ ]:
# How many website per country?   (count distinct websites)
sources = full_data_sources
websites_per_country = sources.groupby(by=['country'])['website'].nunique()
websites_per_country = pd.DataFrame(websites_per_country)
websites_per_country = websites_per_country.reset_index()
#websites_per_country

In [ ]:
internet_users = data[['country','Internet_users']]
#internet_users

In [ ]:
websites_userPercentage = pd.merge(websites_per_country, internet_users, left_on='country',right_on='country')

In [ ]:
websites_userPercentage

In [ ]:
# Create a trace
trace = go.Scatter(
    x = websites_userPercentage.website,
    y = websites_userPercentage.Internet_users,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-scatter')


In [ ]:
trace1 = go.Bar(
    x=websites_userPercentage.country,
    y=websites_userPercentage.website,
    name='number of unique website'
)
trace2 = go.Bar(
    x=websites_userPercentage.country,
    y=websites_userPercentage.Internet_users,
    name='internet usage',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Double Y Axis Example',
    yaxis=dict(
        title='yaxis title'
    ),
    yaxis2=dict(
        title='yaxis2 title',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)

py.iplot(data, filename='double_bar', layout=layout)

#fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')